In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.1.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 800/1000 [=======================>......] - ETA: 0s - loss: 1.3444 - accuracy: 0.6212
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 932us/sample - loss: 1.2024 - accuracy: 0.6630 - val_loss: 0.7150 - val_accuracy: 0.7770
Epoch 2/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.4335 - accuracy: 0.8780
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 488us/sample - loss: 0.4334 - accuracy: 0.8780 - val_loss: 0.5199 - val_accuracy: 0.8410
Epoch 3/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2807 - accuracy: 0.9252
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 365us/sample - loss: 0.2832 - accuracy: 0.9260 - val_loss: 0.5014 - val_accuracy: 0.8450
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2055 - accuracy: 0.9463
Epoch

In [5]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3262 - accuracy: 0.0640
훈련되지 않은 모델의 정확도:  6.40%


In [6]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.3877 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


In [7]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [8]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [9]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.5027 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


In [10]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.5027 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


In [11]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 624us/sample - loss: 1.1679 - accuracy: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 239us/sample - loss: 0.4377 - accuracy: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 273us/sample - loss: 0.2905 - accuracy: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 211us/sample - loss: 0.2072 - accuracy: 0.9470
Epoch 5/5
1000/1000 [==============================] - 0s 256us/sample - loss: 0.1560 - accuracy: 0.9660


In [12]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4474 - accuracy: 0.8590
복원된 모델의 정확도: 85.90%


In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 701us/sample - loss: 1.1406 - accuracy: 0.6850
Epoch 2/5
1000/1000 [==============================] - 0s 262us/sample - loss: 0.4277 - accuracy: 0.8700
Epoch 3/5
1000/1000 [==============================] - 0s 253us/sample - loss: 0.2996 - accuracy: 0.9190
Epoch 4/5
1000/1000 [==============================] - 0s 256us/sample - loss: 0.2092 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 232us/sample - loss: 0.1613 - accuracy: 0.9680


In [15]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.experimental' has no attribute 'export_saved_model'

In [16]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

AttributeError: module 'tensorflow_core.keras.experimental' has no attribute 'load_from_saved_model'

In [17]:
model.predict(test_images).shape


(1000, 10)

In [18]:
# 이 모델을 평가하려면 그전에 컴파일해야 합니다.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=model.optimizer, # 복원된 옵티마이저를 사용합니다.
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4474 - accuracy: 0.8590
복원된 모델의 정확도: 85.90%
